In [95]:
import json
  
# Opening JSON file
f = open('xcm-acala-interlay.json')
  
# returns JSON object as 
# a dictionary
acala_interlay = json.load(f)
len(acala_interlay)

326

In [96]:
import json
  
# Opening JSON file
f = open('xcm-moonbeam-interlay.json')
  
# returns JSON object as 
# a dictionary
moonbeam_interlay = json.load(f)
len(moonbeam_interlay)

271

In [59]:
acala_startblock = 1638215
interlay_startblock = 975127

ibtc_asset_code = '{"Token":"IBTC"}'

In [74]:
acala_interlay[0]

{'msgHash': '0x85416c55ac8d37412dd8f6150c1a41bf055891d1d02481a16a206443c0b2a6c2',
 'extrinsicHash': '0x05c9452f4ab58509c8419b41249cec3ae79272f6991b469a09eaa212bc08c3e9',
 'extrinsicID': '1639146-7',
 'chainID': 2000,
 'chainIDDest': 2032,
 'blockNumber': '1639146',
 'fromAddress': '0xc65841308cad09abfa6c62f505e91e980842039aefdd1f7cafd1ee47dd522016',
 'destAddress': '0xc65841308cad09abfa6c62f505e91e980842039aefdd1f7cafd1ee47dd522016',
 'sectionMethod': 'xTokens:TransferredMultiAssets',
 'asset': '{"Token":"IBTC"}',
 'rawAsset': '{"Token":"IBTC"}',
 'nativeAssetChain': '{"Token":"IBTC"}~2032',
 'blockNumberDest': 976075,
 'sourceTS': 1660442112,
 'destTS': 1660442130,
 'amountSent': 0.00308416,
 'amountReceived': 0.00308344,
 'status': None,
 'relayChain': 'polkadot',
 'incomplete': 0,
 'amountSentUSD': 0,
 'amountReceivedUSD': 0,
 'chainName': 'Acala',
 'id': 'acala',
 'idDest': 'interlay',
 'chainDestName': 'Interlay'}

# Total XCM transfers since start of attack

In [97]:
acala_interlay_since_attack = [x for x in acala_interlay if int(x["blockNumberDest"] or 0) >= interlay_startblock and x["asset"] == ibtc_asset_code]

In [98]:
moonbeam_interlay_since_attack = [x for x in moonbeam_interlay if int(x["blockNumberDest"] or 0) >= interlay_startblock and x["asset"] == ibtc_asset_code]

In [99]:
len(acala_interlay_since_attack)

9

In [109]:
acala_interlay_since_attack

[{'msgHash': '0x85416c55ac8d37412dd8f6150c1a41bf055891d1d02481a16a206443c0b2a6c2',
  'extrinsicHash': '0x05c9452f4ab58509c8419b41249cec3ae79272f6991b469a09eaa212bc08c3e9',
  'extrinsicID': '1639146-7',
  'chainID': 2000,
  'chainIDDest': 2032,
  'blockNumber': '1639146',
  'fromAddress': '0xc65841308cad09abfa6c62f505e91e980842039aefdd1f7cafd1ee47dd522016',
  'destAddress': '0xc65841308cad09abfa6c62f505e91e980842039aefdd1f7cafd1ee47dd522016',
  'sectionMethod': 'xTokens:TransferredMultiAssets',
  'asset': '{"Token":"IBTC"}',
  'rawAsset': '{"Token":"IBTC"}',
  'nativeAssetChain': '{"Token":"IBTC"}~2032',
  'blockNumberDest': 976075,
  'sourceTS': 1660442112,
  'destTS': 1660442130,
  'amountSent': 0.00308416,
  'amountReceived': 0.00308344,
  'status': None,
  'relayChain': 'polkadot',
  'incomplete': 0,
  'amountSentUSD': 0,
  'amountReceivedUSD': 0,
  'chainName': 'Acala',
  'id': 'acala',
  'idDest': 'interlay',
  'chainDestName': 'Interlay'},
 {'msgHash': '0x4d7bc466ecb879e5510d3643

In [100]:
len(moonbeam_interlay_since_attack)

83

## iBTC Sent via XCM 

### Acala

In [90]:
iBTC_sent_by_acala = sum([x["amountSent"] for x in acala_interlay_since_attack])
iBTC_sent_by_acala

3.6236798300000004

In [91]:
iBTC_received_from_acala =  sum([x["amountReceived"] for x in acala_interlay_since_attack])
iBTC_received_from_acala

3.6236733500000002

### Moonbeam

In [93]:
iBTC_sent_by_moonbeam = sum([x["amountSent"] for x in moonbeam_interlay_since_attack])
iBTC_sent_by_moonbeam

2.31959467

In [94]:
iBTC_received_from_moonbeam =  sum([x["amountReceived"] for x in moonbeam_interlay_since_attack])
iBTC_received_from_moonbeam

2.3195349100000024

### Accounts on Interlay that received iBTC via XCM after attack started

In [105]:
accounts_from_acala = set([x["destAddress"] for x in acala_interlay_since_attack])#
len(accounts_from_acala)

6

In [107]:
accounts_from_acala

{'0x78a2131d96076c4fd0cf7e06239934881bc57d197b16ac12e1c1554759c4d65e',
 '0xa07e831679493815f8270f903d48495e0761757575e0a2345579ebce9a694e1f',
 '0xae3a2acf23a65b9d048e3835c84121d39698c88d38a448e7f200ed004ff56d60',
 '0xb8bc052ea768082988fab19385c73e485d4c2bc36b921b7516544b0a2824bf46',
 '0xc65841308cad09abfa6c62f505e91e980842039aefdd1f7cafd1ee47dd522016',
 '0xd09a1884c8ca6641e644447f160be702de9c6748da6b51c553d6ff35cd8e4e08'}

In [106]:
accounts_from_moonbeam = set([x["destAddress"] for x in moonbeam_interlay_since_attack])
len(accounts_from_moonbeam)

17

In [108]:
accounts_from_moonbeam

{'0x06acb4395afa4f5cbaf90376b04e657bd9b7c0267a806dc5ce09c97cc2562a25',
 '0x26abeb5aa3dad23b0664de001d2cd5ba36bcc395ba7e006327f4f8f2afbac07b',
 '0x542f6a676208c8d377ed246b04e031e2f97d4e190e6045fd6060840a87545d2d',
 '0x5835e670318b79c3c79ebb94b55171d83b031dfa29c761c88550613cb911d33e',
 '0x64c240c47592bcfef2e48b1da1b04d0022e03162520b9cd84eaea7717b15905d',
 '0x704af31b65f68c7d6a7c2c11f287da2eb1d2e61a8b8db214ea3186175f33532a',
 '0x764789e3ae824b2412cd813d8cee45893e951ded7ef93b68940c5ed7b794ac38',
 '0x9a8260623bc5831d8c2b343584a77007782cf231f8063e2caa2abb4ff686c466',
 '0xa07e831679493815f8270f903d48495e0761757575e0a2345579ebce9a694e1f',
 '0xae3a2acf23a65b9d048e3835c84121d39698c88d38a448e7f200ed004ff56d60',
 '0xb881c3e7bbaed7522c8b906a9a63e6572f02f87ae76236a8f06124102061765d',
 '0xb8bc052ea768082988fab19385c73e485d4c2bc36b921b7516544b0a2824bf46',
 '0xc65841308cad09abfa6c62f505e91e980842039aefdd1f7cafd1ee47dd522016',
 '0xc89196fbcc06e76a7f9ae6a570508ec5c9f6966a4a4ccc1e3e4b250f55901952',
 '0xce